

#This tool sets up a text retrieval and question-answering system using Streamlit, LangChain, and OpenAI APIs, leveraging embeddings and FAISS indexing for efficient searches.

This is a basic learing tool for the retrival of info on a given Query, was aiming for embedding with OpenAIEmbeddings() and vector indexing with vectorindex_openai but could'nt

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00


In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.8/355.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 15.3 MB/s eta 0:00:00


In [ ]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.3 MB/s eta 0:00:00


In [ ]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain_community.llms import OpenAI # Import OpenAI from the correct module
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


In [ ]:
#load openAI api key
os.environ['OPENAI_API_KEY'] = 'OPENAI API KEY'

In [ ]:
# Initialise LLM with required params
llm = OpenAI(temperature=0.9, max_tokens=500)

### (1) Load data

In [ ]:
!pip install unstructured

  Using cached unstructured-0.14.10-py3-none-any.whl (2.1 MB)
  Using cached python_magic-0.4.27-py2.py3-none-any.whl (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.6 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=d9922ef35f000c40f03f61836e71e3e9e272d84ca8f08393ed089784a7d286d5
  Stored in directory: 

In [ ]:
!pip3 install unstructured libmagic python-magic python-magic-bin

  Using cached libmagic-1.0.tar.gz (3.7 kB)
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement python-magic-bin (from versions: none)
ERROR: No matching distribution found for python-magic-bin


In [ ]:
from langchain.document_loaders import UnstructuredURLLoader

In [ ]:
loaders = UnstructuredURLLoader(urls=[
    "https://www.deccanherald.com/india/no-disagreements-between-modi-putin-at-meet-kremlin-3100152",
    "https://www.thehindu.com/news/international/indias-relationship-with-russia-gives-it-ability-to-urge-russian-president-putin-to-end-war-in-ukraine-white-house/article68387851.ece",
    "https://economictimes.indiatimes.com/news/india/we-never-wanted-them-russia-on-indians-in-russian-army/articleshow/111638059.cms?from=mdr"
])
data = loaders.load()
len(data)

3

In [ ]:
data[2]

Document(metadata={'source': 'https://economictimes.indiatimes.com/news/india/we-never-wanted-them-russia-on-indians-in-russian-army/articleshow/111638059.cms?from=mdr'}, page_content='Benchmarks\n\nNifty\n\n24,324.45\n\n108.75\n\nFEATURED FUNDS★★★★★Canara Robeco Flexi Cap Fund Regular-Growth5Y Return19.56 %\n                Invest Now\n\nFEATURED FUNDS★★★★★Canara Robeco Infrastructure Fund Regular-Growth5Y Return29.8 %\n                Invest Now\n\nNews\n\nEnglish Edition\n\nEnglish Editionहिन्दीગુજરાતીमराठीবাংলাಕನ್ನಡമലയാളംதமிழ்తెలుగు\n\nToday\'s ePaper\n\nSubscribe\n\nSign In\n\nMonsoon Offer is Live\n\nHome\n\nBUDGET\'24\n\nETPrime\n\nMarkets\n\nMarket Data\n\nNews\n\nIndustry\n\nRise\n\nPolitics\n\nWealth\n\nMF\n\nTech\n\nCareers\n\nOpinion\n\nNRI\n\nPanache\n\nMore Menu\n\nIndia\n\nDecoded\n\nWeb Stories\n\nMorning Brief Podcast\n\nNewsblogs\n\nEconomy\n\nAgriculture\n\nFinance\n\nForeign Trade\n\nIndicators\n\nInfrastructure\n\nPolicy\n\nIndustry\n\nPolitics\n\nET Explains\n\nMo

### (2) Split data to create chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)

In [ ]:
data[0]

Document(metadata={'source': 'https://www.deccanherald.com/india/no-disagreements-between-modi-putin-at-meet-kremlin-3100152'}, page_content='Newsletters\n\nePaper\n\nSign in\n\nHome\n\nIndia\n\nKarnataka\n\nOpinion\n\nWorld\n\nBusiness\n\nSports\n\nEntertainment\n\nVideo\n\nDH Specials\n\nNews Shots\n\nExplainers\n\nBengaluru\n\nScience\n\nTrending\n\nPhotos\n\nBrandspot\n\nNewsletters\n\nHome\n\nNews Shots\n\nTrending\n\nMenu\n\nADVERTISEMENT\n\nADVERTISEMENT\n\nADVERTISEMENT\n\nHome\n\nindia\n\nNo disagreements between Modi-Putin at meet: Kremlin\n\nKremlin spokesman Dmitry Peskov told reporters a decision to cancel a session of talks between Russian and Indian delegations was taken for scheduling reasons and because the entire agenda had been covered.\n\nReuters\n\nLast Updated : 10 July 2024, 11:48 IST\n\nFollow Us :\n\nComments\n\nMoscow: The Kremlin denied on Wednesday that there had been any disagreement between President Vladimir Putin and Indian Prime Minister Narendra Modi a

In [ ]:
len(docs)

36

In [ ]:
docs[1].metadata

{'source': 'https://www.deccanherald.com/india/no-disagreements-between-modi-putin-at-meet-kremlin-3100152'}

In [ ]:
len(data)

3

### (3) Create embeddings for these chunks and save them to FAISS index

In [ ]:
pip install tiktoken

In [ ]:
!pip install faiss-cpu

In [ ]:
import time
import faiss
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
pip install transformers sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
texts = [doc.page_content for doc in data]
# Generate embeddings
embeddings = model.encode(texts)

# Display the embeddings
print(embeddings)

[[-0.07307539  0.06505676  0.02221289 ... -0.0398837  -0.05369519
  -0.03286352]
 [-0.06298874 -0.02311068 -0.07275425 ... -0.07358043 -0.08779863
  -0.03324759]
 [ 0.01155523 -0.09280917 -0.03517363 ... -0.16054083 -0.01226814
   0.02720246]]


In [ ]:
#embeddings = OpenAIEmbeddings()

In [ ]:
import numpy as np

In [ ]:
# Convert embeddings to a numpy array
embeddings = np.array(embeddings)

# Create a FAISS index
dimension = embeddings.shape[1]  # Dimension of the embeddings
index = faiss.IndexFlatL2(dimension)  # Use L2 (Euclidean) distance

# Add embeddings to the index
index.add(embeddings)

In [ ]:
# Check the number of vectors in the index
print(f"Number of vectors in the index: {index.ntotal}")

Number of vectors in the index: 3


In [ ]:
from transformers import pipeline

In [ ]:
# Generate embeddings using SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(texts)

# Convert embeddings to a numpy array
embeddings = np.array(embeddings)

# Create a FAISS index
dimension = embeddings.shape[1]  # Dimension of the embeddings
index = faiss.IndexFlatL2(dimension)  # Use L2 (Euclidean) distance

# Add embeddings to the index
index.add(embeddings)

print(f"Number of vectors in the index: {index.ntotal}")

Number of vectors in the index: 3


In [ ]:

def find_answer(question, model, index, texts, k=2):
    # Generate embedding for the question
    question_embedding = model.encode([question])[0].reshape(1, -1)

    # Search the FAISS index
    distances, indices = index.search(question_embedding, k)

    # Retrieve the most similar documents
    nearest_docs = [texts[i] for i in indices[0]]

    return nearest_docs, distances

In [ ]:
# Load a pre-trained question-answering model
qa_pipeline = pipeline("question-answering")

def generate_answer(question, texts, qa_pipeline):
    # Use the QA model to generate an answer from the retrieved documents
    context = " ".join(texts)
    result = qa_pipeline(question=question, context=context)
    return result['answer']

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
# Example question

question1 = "We are on the same side with the"

# Find the nearest documents
nearest_docs, distances = find_answer(question1, model, index, texts)

# Generate the answer using the QA pipeline
answer = generate_answer(question1, nearest_docs, qa_pipeline)

# print(f"Nearest documents: {nearest_docs}")
print(f"Distances: {distances}")
print(f"nearest_docs: {nearest_docs}")

print(f"Question: {question1}")

print(f"Answer: {answer}")


Distances: [[1.7347705 1.9129475]]
nearest_docs: ['Newsletters\n\nePaper\n\nSign in\n\nHome\n\nIndia\n\nKarnataka\n\nOpinion\n\nWorld\n\nBusiness\n\nSports\n\nEntertainment\n\nVideo\n\nDH Specials\n\nNews Shots\n\nExplainers\n\nBengaluru\n\nScience\n\nTrending\n\nPhotos\n\nBrandspot\n\nNewsletters\n\nHome\n\nNews Shots\n\nTrending\n\nMenu\n\nADVERTISEMENT\n\nADVERTISEMENT\n\nADVERTISEMENT\n\nHome\n\nindia\n\nNo disagreements between Modi-Putin at meet: Kremlin\n\nKremlin spokesman Dmitry Peskov told reporters a decision to cancel a session of talks between Russian and Indian delegations was taken for scheduling reasons and because the entire agenda had been covered.\n\nReuters\n\nLast Updated : 10 July 2024, 11:48 IST\n\nFollow Us :\n\nComments\n\nMoscow: The Kremlin denied on Wednesday that there had been any disagreement between President Vladimir Putin and Indian Prime Minister Narendra Modi at their talks in Moscow this week.\n\nModi delivered an implicit rebuke to Putin on Tuesday

In [ ]:
# "https://www.deccanherald.com/india/no-disagreements-between-modi-putin-at-meet-kremlin-3100152",
# "https://www.thehindu.com/news/international/indias-relationship-with-russia-gives-it-ability-to-urge-russian-president-putin-to-end-war-in-ukraine-white-house/article68387851.ece",
# "https://economictimes.indiatimes.com/news/india/we-never-wanted-them-russia-on-indians-in-russian-army/articleshow/111638059.cms?from=mdr"